<a href="https://colab.research.google.com/github/Donglllai/Supply-chain/blob/main/Biopherma0212.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install gurobipy
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from gurobipy import Model, GRB

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from gurobipy import Model, GRB, quicksum


np.random.seed(42)


num_samples = 100
mean_rates = np.array([1.1, 0.9, 1.3, 1.2, 0.8, 1])
cov_matrix = np.array([
    [0.02, 0.01, 0.01, 0.00, 0.00, 0.00],
    [0.01, 0.03, 0.00, 0.01, 0.00, 0.00],
    [0.01, 0.00, 0.04, 0.01, 0.00, 0.00],
    [0.00, 0.01, 0.01, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.03, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
])
exchange_rate_samples = multivariate_normal.rvs(mean_rates, cov_matrix, size=num_samples)

# factory data
factories = ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.']
pcosts = pd.DataFrame({
    'plant': factories,
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [5, 13, 3, 4, 6, 5],
    'fc_r': [5, 13, 3, 4, 6, 5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

caps = pd.DataFrame({'plant': factories, 'cap': [18, 45, 18, 10, 30, 22]})
caps.set_index('plant', inplace=True)

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [7, 15, 5, 7, 3, 18],
    'd_r': [7, 12, 3, 8, 3, 17],
})
demand.set_index('from', inplace=True)


factory_open_matrix = np.zeros((num_samples, len(factories)))

factory_open_counts = pd.DataFrame(0, index=factories, columns=['Open Count'])

for i in range(num_samples):

    reindx = mean_rates / exchange_rate_samples[i]
    pcosts_rev = pcosts.values * reindx.reshape(-1, 1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns, index=pcosts.index)

    model = Model("MinimizeCost")

    dec_plant = {plant: model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{plant}") for plant in factories}
    dec_h = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in factories for j in demand.index}
    dec_r = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in factories for j in demand.index}

    # constraint 1
    for plant in factories:
        model.addConstr(
            quicksum(dec_h[plant, j] + dec_r[plant, j] for j in demand.index) <= dec_plant[plant] * caps.loc[plant, 'cap'],
            name=f"Capacity_{plant}"
        )

    # constraint 2
    for j in demand.index:
        model.addConstr(
            quicksum(dec_h[i, j] for i in factories) >= demand.loc[j, 'd_h'],
            name=f"Demand_HighCal_{j}"
        )
        model.addConstr(
            quicksum(dec_r[i, j] for i in factories) >= demand.loc[j, 'd_r'],
            name=f"Demand_Relax_{j}"
        )

    # target: minimize cost
    total_cost = quicksum(
        dec_plant[i] * pcosts_rev.loc[i, 'fc_p'] for i in factories
    ) + quicksum(
        (pcosts_rev.loc[i, 'rm_h'] + pcosts_rev.loc[i, 'pc_h']) * dec_h[i, j]
        + (pcosts_rev.loc[i, 'rm_r'] + pcosts_rev.loc[i, 'pc_r']) * dec_r[i, j]
        for i in factories for j in demand.index
    )
    model.setObjective(total_cost, GRB.MINIMIZE)
    model.Params.OutputFlag = 0
    model.optimize()


    for idx, plant in enumerate(factories):
        factory_open_matrix[i, idx] = int(dec_plant[plant].x > 0.5)
    for plant in factories:
        if dec_plant[plant].x > 0.5:
            factory_open_counts.loc[plant, 'Open Count'] += 1

# output
print(factory_open_counts)

factory_open_df = pd.DataFrame(factory_open_matrix, columns=factories)

print(factory_open_df)



         Open Count
Brazil           79
Germany         100
India            92
Japan            54
Mexico           72
U.S.             85
    Brazil  Germany  India  Japan  Mexico  U.S.
0      1.0      1.0    1.0    0.0     1.0   0.0
1      0.0      1.0    1.0    0.0     1.0   1.0
2      1.0      1.0    1.0    1.0     1.0   1.0
3      1.0      1.0    1.0    1.0     0.0   1.0
4      0.0      1.0    1.0    1.0     1.0   1.0
..     ...      ...    ...    ...     ...   ...
95     0.0      1.0    0.0    1.0     1.0   1.0
96     1.0      1.0    1.0    1.0     1.0   1.0
97     0.0      1.0    0.0    1.0     1.0   1.0
98     1.0      1.0    1.0    0.0     1.0   1.0
99     1.0      1.0    1.0    1.0     0.0   1.0

[100 rows x 6 columns]


In [10]:
import pandas as pd
import numpy as np


exchange_rate_samples_df = pd.DataFrame(exchange_rate_samples,columns=factories)

exchange_rate_correlation = exchange_rate_samples_df.corr()

# negative correlated in exchange rate <-0.1
negative_correlated_pairs = [(i, j) for i in factories for j in factories
                             if i != j and exchange_rate_correlation.loc[i, j] < -0.1]

print("negative correlated:", negative_correlated_pairs)
# positive correlated in exchange rate>0.1
postive_correlated_pairs = [(i, j) for i in factories for j in factories
                             if i != j and exchange_rate_correlation.loc[i, j] > 0.1]

print("postive correlated", postive_correlated_pairs)


negative correlated: []
postive correlated [('Brazil', 'Germany'), ('Brazil', 'India'), ('Germany', 'Brazil'), ('Germany', 'India'), ('Germany', 'Japan'), ('India', 'Brazil'), ('India', 'Germany'), ('India', 'Japan'), ('Japan', 'Germany'), ('Japan', 'India')]


In [11]:
factory_open_df = pd.DataFrame(factory_open_matrix, columns=factories)

# calculate correlation in factory opening
correlation_matrix = factory_open_df.corr()

# find factory that have correlation bigger than 0.1
correlation_filtered = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
correlation_filtered = correlation_filtered.stack().reset_index()
correlation_filtered.columns = ['Factory_1', 'Factory_2', 'Correlation']
correlation_filtered = correlation_filtered[correlation_filtered['Correlation'] > 0.1]

# result
print(correlation_filtered)

  Factory_1 Factory_2  Correlation
0    Brazil     India     0.119457
1    Brazil     Japan     0.115270
8     Japan      U.S.     0.455150


Since Brazil India is the only pair that have both high correlation in factory open and exchange rate. Thus, the pair we want to find is **"Brazil India"**

In [12]:
plant_open_frequencies = factory_open_counts['Open Count']/100

def generate_strategy(threshold, frequencies):
    return (frequencies > threshold).astype(int)

strategies = {
    "High Robustness": generate_strategy(0.8, plant_open_frequencies),
    "Balanced Robustness": generate_strategy(0.6, plant_open_frequencies),
    "Flexible": generate_strategy(0.4, plant_open_frequencies),
}

def evaluate_strategy(strategy, rate_samples):
    costs = []
    for rates in rate_samples:

        reindx = mean_rates / rates
        adjusted_costs = pcosts.values * reindx.reshape(-1, 1)
        adjusted_costs = pd.DataFrame(adjusted_costs, columns=pcosts.columns, index=pcosts.index)


        fixed_cost = sum(adjusted_costs.loc[plant, 'fc_p'] * strategy[plant] for plant in factories)


        variable_cost = sum(adjusted_costs.loc[plant, 'rm_h'] + adjusted_costs.loc[plant, 'pc_h']
                            + adjusted_costs.loc[plant, 'rm_r'] + adjusted_costs.loc[plant, 'pc_r']
                            for plant in factories if strategy[plant] == 1)

        total_cost = fixed_cost + variable_cost
        costs.append(total_cost)

    return {
        "mean_cost": np.mean(costs),
        "std_cost": np.std(costs)
    }

strategy_evaluations = {
    name: evaluate_strategy(strategy, exchange_rate_samples)
    for name, strategy in strategies.items()
}
recommended_strategy = min(strategy_evaluations, key=lambda s: (strategy_evaluations[s]['mean_cost'], strategy_evaluations[s]['std_cost']))
print("Plant Open Frequencies:", plant_open_frequencies)
print("Strategies:", strategies)
print("Strategy Evaluations:", strategy_evaluations)
print("Recommended Strategy:", recommended_strategy)


Plant Open Frequencies: Brazil     0.79
Germany    1.00
India      0.92
Japan      0.54
Mexico     0.72
U.S.       0.85
Name: Open Count, dtype: float64
Strategies: {'High Robustness': Brazil     0
Germany    1
India      1
Japan      0
Mexico     0
U.S.       1
Name: Open Count, dtype: int64, 'Balanced Robustness': Brazil     1
Germany    1
India      1
Japan      0
Mexico     1
U.S.       1
Name: Open Count, dtype: int64, 'Flexible': Brazil     1
Germany    1
India      1
Japan      1
Mexico     1
U.S.       1
Name: Open Count, dtype: int64}
Strategy Evaluations: {'High Robustness': {'mean_cost': 146.16644929351762, 'std_cost': 16.203897587590596}, 'Balanced Robustness': {'mean_cost': 239.38328422200982, 'std_cost': 23.169914322905097}, 'Flexible': {'mean_cost': 276.778390282508, 'std_cost': 25.139254913631927}}
Recommended Strategy: High Robustness
